<a href="https://colab.research.google.com/github/sakagarwal/Appetizers4Days/blob/master/%5BPrivate_Preview%5D_Wide_%26_Deep_on_Managed_Pipelines.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Copyright 2018 Google LLC.

Licensed under the Apache License, Version 2.0 (the "License");

# Wide & Deep on Managed Pipelines

This notebook showcases how to run the Wide & Deep algorithm with Managed Pipelines on Vertex AI using the built-in pipelines from the SDK.

The notebook is organized as follows:

- [Setup environment](#setup): Install depdendencies, authenticate, and configure GCP project used in the notebook.
- [Define training inputs](#define-training-inputs): Define training inputs such as data location, data split type, feature transformations etc.
- [Run a CustomJob](#run-custom-job): Set CustomJob configurations such as machine type and hyperparameters and train a model on Vertex AI Managed Pipelines.
- [Run a HyperparameterTuningJob](#run-hyperparameter-tuning-job): Set HyperparameterTuningJob configurations such as machine type and hyperparameters and train a model on Vertex AI Managed Pipelines. 




<a name="setup"></a>

##Setup

### Install dependencies

In [ ]:
# Depending on the environment, this might throw a
# pip dependency resolver error. Please ignore it.
!pip3 install -U google-cloud-pipeline-components -q

In [ ]:
# Restart the kernel after pip installs. This can take a minute. 
import IPython
app = IPython.Application.instance()
app.kernel.do_shutdown(True)

In [ ]:
# Import required modules
import json
import os
import sys
import tensorflow as tf
import uuid
from google.cloud import aiplatform
from google.cloud import storage
from google_cloud_pipeline_components.experimental.automl.tabular import utils
from typing import Any, Dict, List

### Configure your GCP project

In [ ]:
GCP_PROJECT = 'cloud-automl-tables' #@param {type:'string'}
GCP_REGION = 'us-central1' #@param {type:'string'}

### Authenticate your GCP account

In [ ]:
if 'google.colab' in sys.modules:
  if 'USE_AUTH_EPHEM' in os.environ:
    # revert to the old colab authentication module to prevent a bug.
    del os.environ['USE_AUTH_EPHEM']
  from google.colab import auth as google_auth
  google_auth.authenticate_user()
  !gcloud config set project {GCP_PROJECT}

aiplatform.init(
    project=GCP_PROJECT,
    location=GCP_REGION
)

### Create GCS base path

All training related files (TF model checkpoint, TensorBoard file, etc) will be saved to the GCS bucket. The pipeline will not clean up the files since some of them might be useful for you, **please make sure to clean up the files**. For easy cleanup, you can set [GCS bucket level TTL](https://cloud.google.com/storage/docs/lifecycle).


In [ ]:
GCS_BASE_PATH = '' # @param {type:'string'}
generate_gcs_base_path = True # @param {type:'boolean'}
if generate_gcs_base_path:
  bucket_name = f'gs://test-{uuid.uuid4()}'
  !gsutil mb -p {GCP_PROJECT} -l {GCP_REGION} {bucket_name}
  
  # Set GCS bucket object TTL to 7 days
  !echo '{"rule":[{"action": {"type": "Delete"},"condition": {"age": 7}}]}' > gcs_lifecycle.tmp
  !gsutil lifecycle set gcs_lifecycle.tmp {bucket_name}
  !rm gcs_lifecycle.tmp
  
  GCS_BASE_PATH = bucket_name + '/mp_notebook'
  print(f'changed gcs_base_path to {GCS_BASE_PATH} due to generate_gcs_base_path is True')


### Enable APIs (one-time setup)
[Enable the following APIs: Vertex AI APIs, Compute Engine APIs, Cloud Storage and Dataflow.](https://pantheon.corp.google.com/apis/enableflow?apiid=aiplatform.googleapis.com,dataflow,compute_component,storage-component.googleapis.com&mods=-ai_platform_fake_service)


<a name="define-training-inputs"></a>

## Define training inputs



### Define helper functions

In [ ]:
def write_text_to_file(text, filepath):
  with tf.io.gfile.GFile(filepath, 'w') as f:
    f.write(text)

def get_task_detail(task_details: List[Dict[str, Any]], task_name: str) -> List[Dict[str, Any]]:
  """Returns the task details for a specified task."""
  for task_detail in task_details:
    if task_detail.task_name == task_name:
      return task_detail

def get_model_artifacts_path(task_details, task_name):
  """Returns GCS uri to a file with a path to the training artifacts."""
  task = get_task_detail(task_details, task_name)
  # model_artifact_uri points to a file in GCS that contains the model URI
  return task.outputs['unmanaged_container_model'].artifacts[0].uri

def get_model_uri(task_details):
  """Returns a link to the Vertex Model."""
  task = get_task_detail(task_details, 'model-upload')
  # in format https://<location>-aiplatform.googleapis.com/v1/projects/<project_number>/locations/<location>/models/<model_id>
  model_id = task.outputs['model'].artifacts[0].uri.split('/')[-1]
  return f'https://console.cloud.google.com/vertex-ai/locations/{GCP_REGION}/models/{model_id}?project={GCP_PROJECT}'


### Training inputs

#### The following parameters can be set:

##### Pipeline parameters

- `project`: The GCP project that runs the pipeline components.
- `location`: The GCP region that runs the pipeline components.
- `root_dir`: The root GCS directory for the pipeline components.
- `target_column`: The target column name.
- `prediction_type`: The type of prediction the model is to produce.
  'classification' or 'regression'.
- `transform_config`: The path to a GCS file containing the transformations to apply.
data_source_csv_filenames: The CSV data source.
- `data_source_bigquery_table_path`: The BigQuery data source.
- `predefined_split_key`: The predefined_split column name.
- `timestamp_split_key`: The timestamp_split column name.
- `stratified_split_key`: The stratified_split column name.
- `training_fraction`: The training fraction.
- `validation_fraction`: The validation fraction.
- `test_fraction`: The test fraction.
- `weight_column`: The weight column name.
- `stats_and_example_gen_dataflow_machine_type`: The dataflow machine type for
  stats_and_example_gen component.
- `stats_and_example_gen_dataflow_max_num_workers`: The max number of Dataflow
  workers for stats_and_example_gen component.
- `stats_and_example_gen_dataflow_disk_size_gb`: Dataflow worker's disk size in
  GB for stats_and_example_gen component.
- `transform_dataflow_machine_type`: The dataflow machine type for transform
  component.
- `transform_dataflow_max_num_workers`: The max number of Dataflow workers for
  transform component.
- `transform_dataflow_disk_size_gb`: Dataflow worker's disk size in GB for
  transform component.
- `training_machine_spec`: The machine spec for trainer component. See https://cloud.google.com/compute/docs/machine-types for options.
- `training_replica_count`: The replica count for the trainer component.
- `run_evaluation`: Whether to run evaluation steps during training.
- `evaluation_batch_predict_machine_type`: The prediction server machine type for batch predict components during evaluation.
- `evaluation_batch_predict_starting_replica_count`: The initial number of prediction server for batch predict components during evaluation.
- `evaluation_batch_predict_max_replica_count`: The max number of prediction server for batch predict components during evaluation.
- `evaluation_dataflow_machine_type`: The dataflow machine type for evaluation components.
- `evaluation_dataflow_max_num_workers`: The max number of Dataflow workers for evaluation components.
- `evaluation_dataflow_disk_size_gb`: Dataflow worker's disk size in GB for evaluation components.
- `dataflow_service_account`: Custom service account to run dataflow jobs.
- `dataflow_subnetwork`: Dataflow's fully qualified subnetwork name, when empty
  the default
  subnetwork will be used. Example:
    https://cloud.google.com/dataflow/docs/guides/specifying-networks#example_network_and_subnetwork_specifications
- `dataflow_use_public_ips`: Specifies whether Dataflow workers use public IP
  addresses.
- `encryption_spec_key_name`: The KMS key name.

##### Model hyperparameters
- `learning_rate`: The learning rate used by the linear optimizer.
- `dnn_learning_rate`: The learning rate for training the deep part of the
model.
- `optimizer_type`: The type of optimizer to use. Choices are 'adam', 'ftrl' and 'sgd' for the Adam, FTRL, and Gradient Descent Optimizers, respectively.
- `max_steps`: Number of steps to run the trainer for.
- `max_train_secs`: Amount of time in seconds to run the trainer for.
- `l1_regularization_strength`: L1 regularization strength for optimizer_type='ftrl'.
- `l2_regularization_strength`: L2 regularization strength for
optimizer_type='ftrl'.
- `l2_shrinkage_regularization_strength`: L2 shrinkage regularization strength
for optimizer_type='ftrl'.
- `beta_1`: Beta 1 value for optimizer_type='adam'.
- `beta_2`: Beta 2 value for optimizer_type='adam'.
- `hidden_units`: Hidden layer sizes to use for DNN feature columns, provided in comma-separated layers.
- `use_wide`: If set to True, the categorical columns will be used in the wide
part of the DNN model.
- `embed_categories`: If set to True, the categorical columns will be used
embedded and used in the deep part of the model. Embedding size is the
square root of the column cardinality.
- `dnn_dropout`: The probability we will drop out a given coordinate.
- `dnn_optimizer_type`: The type of optimizer to use for the deep part of the
model. Choices are 'adam', 'ftrl' and 'sgd'. for the Adam, FTRL, and
Gradient Descent Optimizers, respectively.
- `dnn_l1_regularization_strength`: L1 regularization strength for
dnn_optimizer_type='ftrl'.
- `dnn_l2_regularization_strength`: L2 regularization strength for
dnn_optimizer_type='ftrl'.
- `dnn_l2_shrinkage_regularization_strength`: L2 shrinkage regularization
strength for dnn_optimizer_type='ftrl'.
- `dnn_beta_1`: Beta 1 value for dnn_optimizer_type='adam'.
- `dnn_beta_2`: Beta 2 value for dnn_optimizer_type='adam'.
- `enable_profiler`: Enables profiling and saves a trace during evaluation.
- `seed`: Seed to be used for this run.
- `eval_steps`: Number of steps to run evaluation for. If not
specified or negative, it means run evaluation on the whole validation
dataset. If set to 0, it means run evaluation for a fixed number of
samples.
- `batch_size`: Batch size for training.
- `eval_frequency_secs`: Frequency at which evaluation and checkpointing will take place.

HyperparameterTuningJob-specific parameters
- `study_spec_metrics`: List of dictionaries representing metrics to optimize.
The dictionary contains the metric_id, which is reported by the training job, ands the optimization goal of the metric. One of 'minimize' or 'maximize'.
- `study_spec_parameters_override`: List of dictionaries representing parameters to
optimize. The dictionary key is the parameter_id, which is passed to
training job as a command line argument, and the dictionary value is the
parameter specification of the metric.
- `max_trial_count`: The desired total number of trials.
- `parallel_trial_count`: The desired number of trials to run in parallel.
- `algorithm`: Which algorithm to train. Specify 'wide_and_deep' here.
- `max_failed_trial_count`: The number of failed trials that need to be seen before failing the HyperparameterTuningJob. If set to 0, Vertex AI decides how many trials must fail before the whole job fails.
- `study_spec_algorithm`: The search algorithm specified for the study. One of
'ALGORITHM_UNSPECIFIED', 'GRID_SEARCH', or 'RANDOM_SEARCH'.- `study_spec_measurement_selection_type`: Which measurement to use if/when the
service automatically selects the final measurement from previously reported intermediate measurements. One of 'BEST_MEASUREMENT' or 'LAST_MEASUREMENT'.




###Configure dataset

In [ ]:
# CSV data source
csv_filenames = 'gs://automl-tables-us-central1-resources/dataset/safe_driver/train.csv'

# BQ data source
# When using BQ as data source,
# - if BQ is in single-region, GCS root_dir must be in the same single-region.
# - if BQ is multi-region, GCS root_dir must be in the same multi-region or the single-region contained by the multi-region.
# 'big_query_table_path' = 'bq://cloud-automl-tables-public.datasets.safe_driver'

features = ['id', 'ps_ind_01', 'ps_ind_02_cat', 'ps_ind_03', 'ps_ind_04_cat', 'ps_ind_05_cat', 'ps_ind_06_bin', 'ps_ind_07_bin', 'ps_ind_08_bin', 'ps_ind_09_bin', 'ps_ind_10_bin', 'ps_ind_11_bin', 'ps_ind_12_bin', 'ps_ind_13_bin', 'ps_ind_14', 'ps_ind_15', 'ps_ind_16_bin', 'ps_ind_17_bin', 'ps_ind_18_bin', 'ps_reg_01', 'ps_reg_02', 'ps_reg_03', 'ps_car_01_cat', 'ps_car_02_cat', 'ps_car_03_cat', 'ps_car_04_cat', 'ps_car_05_cat', 'ps_car_06_cat', 'ps_car_07_cat', 'ps_car_08_cat', 'ps_car_09_cat', 'ps_car_10_cat', 'ps_car_11_cat', 'ps_car_11', 'ps_car_12', 'ps_car_13', 'ps_car_14', 'ps_car_15', 'ps_calc_01', 'ps_calc_02', 'ps_calc_03', 'ps_calc_04', 'ps_calc_05', 'ps_calc_06', 'ps_calc_07', 'ps_calc_08', 'ps_calc_09', 'ps_calc_10', 'ps_calc_11', 'ps_calc_12', 'ps_calc_13', 'ps_calc_14', 'ps_calc_15_bin', 'ps_calc_16_bin', 'ps_calc_17_bin', 'ps_calc_18_bin', 'ps_calc_19_bin', 'ps_calc_20_bin']

### Configure feature transformation

Transformations can be specified using Feature Transform Engine (FTE) specific configurations. In the following, we provide some sample transform configurations to demonstrate FTE's capabilities:
- Full auto transformations (i.e., `auto_transform_config`): FTE automatically configure a set of built-in transformations for each input column based on its data statistics. 
- Fully specified transformations (i.e., `no_auto_transform_config`): All transformations on input columns are explicitly specified with FTE's built-in transformations. Chaining of multiple transformations on a single column is also supported.
- Mix of auto and explicit transformations (i.e., `mixed_transform_config`).
- Custom transformations (i.e., `transform_config_with_custom_transform`): A mixture of auto and explicit transformations and custom, bring-your-own transform function, where users can define and import their own transform function and use it with FTE's built-in transformations.

In [ ]:
auto_transform_config = {'auto_transforms': features}

no_auto_transform_config = {
    'transforms': [{
        'transform': 'ZScaleTransform',
        'input_column_names': ['ps_reg_01']
    }, {
        'transform': 'ZScaleTransform',
        'input_column_names': ['ps_reg_02']
    }, {
        'transform': 'ZScaleTransform',
        'input_column_names': ['ps_reg_03']
    }, {
        'transform': 'VocabularyTransform',
        'input_column_names': ['ps_ind_10_bin']
    }, {
        'transform': 'VocabularyTransform',
        'input_column_names': ['ps_ind_11_bin']
    }, {
        'transform': 'VocabularyTransform',
        'input_column_names': ['ps_ind_12_bin']
    }, {
        'transform': 'VocabularyTransform',
        'input_column_names': ['target'],
        'output_column_names': ['target']
    }]
}

mixed_transform_config = {
    'auto_transforms': ['ps_reg_01', 'ps_reg_02', 'ps_reg_03'],
    'transforms': [{
        'transform': 'VocabularyTransform',
        'input_column_names': ['ps_ind_10_bin']
    }, {
        'transform': 'VocabularyTransform',
        'input_column_names': ['ps_ind_11_bin']
    }, {
        'transform': 'VocabularyTransform',
        'input_column_names': ['ps_ind_12_bin']
    }]
}

"""
$gsutil cat gs://pvnguyen-us-central1/mp_notebook/custom_transform_fn.py
import tensorflow.compat.v1 as tf


def plus_one_transform(x: tf.SparseTensor) -> tf.SparseTensor:
  return tf.SparseTensor(x.indices, tf.add(x.values, 1), x.dense_shape)
"""
transform_config_with_custom_transform = {
    'auto_transforms': ['ps_reg_02', 'ps_reg_03'],
    'modules': [{
        'transform': 'PlusOneTransform',
        'module_path': 'gs://pvnguyen-us-central1/mp_notebook/custom_transform_fn.py',
        'function_name': 'plus_one_transform'
    }],
    'transforms': [{
        'transform': 'CastToFloatTransform',
        'input_column_names': ['ps_reg_01'],
        'output_column_names': ['ps_reg_01']
    },{
        'transform': 'PlusOneTransform',
        'input_column_names': ['ps_reg_01']
    }, {
        'transform': 'VocabularyTransform',
        'input_column_names': ['ps_ind_10_bin']
    }, {
        'transform': 'VocabularyTransform',
        'input_column_names': ['ps_ind_11_bin']
    }, {
        'transform': 'VocabularyTransform',
        'input_column_names': ['ps_ind_12_bin']
    }]
}

Additional transformations to try out and their sample configurations:

* `DatetimeTransform`:
``` python
# Outputs columns with granular datetime information (year, month, day, etc.).
{
    'transform': 'DatetimeTransform',
    'input_column_names': ['feature_1'],
    'time_format': '%Y-%m-%d'  # time format of input column
}
```

* `LogTransform`:
``` python
# Outputs a column of the element-wise, natural logarithm of our input.
{
    'transform': 'LogTransform',
    'input_column_names': ['feature_1']
}
```

* `ZScaleTransform`:
``` python
# Outputs a z-scale normallized input column.
{
    'transform': 'ZScaleTransform',
    'input_column_names': ['feature_1']
}
```

* `NGramTransform`:
``` python
# Outputs a column containing the vocab lookup incidies of n-grams in our
# input.
{
    'transform': 'NGramTransform',
    'input_column_names': ['feature_1'],
    'min_ngram_size': 1,  # min number of tokens in our n-gram
    'max_ngram_size': 2,  # max number of tokens in our n-gram
    'separator': ' '  # seperator between tokens
  }
```
* `ClipTransform`:
``` python
# Outputs a column where all values < min_value are assigned min_value
# and all columns > max_value are assigned max_value.
{
    'transform': 'ClipTransform',
    'input_column_names': ['col1'],
    'output_column_names': ['col1_clipped'],
    'min_value': 1.,
    'max_value': 10.,
}
```
* `MaxAbsScaleTransform`:
``` python
# Outputs a column where all input elements are divided by abs(max(input)).
{
    'transform': 'MaxAbsScaleTransform',
    'input_column_names': ['col1'],
    'output_column_names': ['col1_max_abs_scaled']
}
```

### Setup training configuration

In [ ]:
prediction_type = 'classification'
target_column = 'target'
optimization_objective = 'maximize-au-roc'

# Fraction split
training_fraction = 0.8
validation_fraction = 0.1
test_fraction = 0.1

# Set feature transformation config
transform_config = auto_transform_config

### VPC-related config

If you need to use a custom Dataflow subnetwork, you can set it through the `dataflow_subnetwork` parameter. The requirements are:
1. `dataflow_subnetwork` must be fully qualified subnetwork name.
   [[reference](https://cloud.google.com/dataflow/docs/guides/specifying-networks#example_network_and_subnetwork_specifications)]
1. The following service accounts must have [Compute Network User role](https://cloud.google.com/compute/docs/access/iam#compute.networkUser) assigned on the specified dataflow subnetwork [[reference](https://cloud.google.com/dataflow/docs/guides/specifying-networks#shared)]:
    1. Compute Engine default service account: PROJECT_NUMBER-compute@developer.gserviceaccount.com
    1. Dataflow service account: service-PROJECT_NUMBER@dataflow-service-producer-prod.iam.gserviceaccount.com

If your project enable VPC-SC, please make sure:

1. The dataflow subnetwork used in VPC-SC is configured properly for Dataflow.
   [[reference](https://cloud.google.com/dataflow/docs/guides/routes-firewall)]
1. `dataflow_use_public_ips` is set to False.



In [ ]:
# Dataflow's fully qualified subnetwork name, when empty the default subnetwork will be used.
dataflow_subnetwork = ''  #@param {type:'string'}
# Specifies whether Dataflow workers use public IP addresses.
dataflow_use_public_ips = True  #@param {type:'boolean'}

<a name="run-custom-job"></a>

## Customize Wide & Deep CustomJob configuration and create pipeline

We will create a Wide & Deep CustomJob pipeline with the following specifications:
- Custom training machine type
- Specify the following hyperparameters: `learning_rate`, `dnn_learning_rate`, `max_steps`, `max_train_secs`

In [ ]:
custom_job_root_dir = os.path.join(GCS_BASE_PATH, 'wide_and_deep_custom_job')

# max_steps and/or max_train_secs must be set. If both are
# specified, training will stop after either condition is met.
# By default, max_train_secs is set to -1.
max_steps = 1000
max_train_secs = -1 

learning_rate = 0.01   #  The learning rate used by the linear optimizer.
dnn_learning_rate = 0.01  # The learning rate for training the deep part of the model.

training_machine_spec =  {
    'machine_type': 'c2-standard-16' # Override for TF chief node
}

transform_config_path = os.path.join(custom_job_root_dir, "transform_config.json")
write_text_to_file(json.dumps(transform_config), transform_config_path)

# If your system does not use Python, you can save the JSON file (`template_path`),
# and use another programming language to submit the pipeline.
custom_job_template_path, custom_job_parameter_values = utils.get_wide_and_deep_trainer_pipeline_and_parameters(
      project=GCP_PROJECT,
      location=GCP_REGION,
      root_dir=custom_job_root_dir,
      max_steps=max_steps,
      max_train_secs=max_train_secs,
      learning_rate=learning_rate,
      dnn_learning_rate=dnn_learning_rate,
      target_column=target_column,
      prediction_type=prediction_type,
      transform_config=transform_config_path,
      training_fraction=training_fraction,
      validation_fraction=validation_fraction,
      test_fraction=test_fraction,
      data_source_csv_filenames=csv_filenames,
      training_machine_spec=training_machine_spec,
      dataflow_use_public_ips=dataflow_use_public_ips,
      dataflow_subnetwork=dataflow_subnetwork,
      run_evaluation=True,
)

custom_job_id = f'automl-tabular-wide-and-deep-{uuid.uuid4()}'
# More info on parameters PipelineJob accepts:
# https://cloud.google.com/vertex-ai/docs/pipelines/run-pipeline#create_a_pipeline_run 
custom_job = aiplatform.PipelineJob(
    display_name=custom_job_id,
    template_path=custom_job_template_path,
    job_id=custom_job_id,
    pipeline_root=custom_job_root_dir,
    parameter_values=custom_job_parameter_values,
    enable_caching=False,
)

custom_job.run()

In [ ]:
# Get model URI
wide_and_deep_trainer_pipeline_task_details = aiplatform.PipelineJob.get(custom_job_id).gca_resource.job_detail.task_details
print('model uri:', get_model_uri(wide_and_deep_trainer_pipeline_task_details))
print('model artifacts:', get_model_artifacts_path(wide_and_deep_trainer_pipeline_task_details, 'automl-tabular-wide-and-deep-trainer'))

<a name="run-hyperparameter-tuning-job"></a>

## Customize Wide & Deep HyperparameterTuningJob configuration and create pipeline

We will create a Wide & Deep HyperparameterTuningJob pipeline with the following specifications:
- Change training machine type

The parameter specs specified in `parameters` can be modified to tune different hyperparameters. The available parameter spec types are [double_value_spec](https://cloud.google.com/vertex-ai/docs/reference/rest/v1beta1/StudySpec#doublevaluespec), [integer_value_spec](https://cloud.google.com/vertex-ai/docs/reference/rest/v1beta1/StudySpec#integervaluespec), [categorical_value_spec](https://cloud.google.com/vertex-ai/docs/reference/rest/v1beta1/StudySpec#integervaluespec), and [discrete_value_spec](https://cloud.google.com/vertex-ai/docs/reference/rest/v1beta1/StudySpec#discretevaluespec). 



In [ ]:
hpt_job_root_dir = os.path.join(GCS_BASE_PATH, 'wide_and_deep_hyperparameter_tuning_job')

training_machine_spec =  {
    'machine_type': 'c2-standard-16' # Override for TF chief node
}

transform_config_path = os.path.join(hpt_job_root_dir, "transform_config.json")
write_text_to_file(json.dumps(transform_config), transform_config_path)

metrics = [{'metric_id': 'loss', 'goal': 'MINIMIZE'}]
parameters = utils.get_wide_and_deep_study_spec_parameters_override()
# max_steps and/or max_train_secs must be set. If both are
# specified, training will stop after either condition is met.
# By default, max_train_secs is set to -1 and max_steps is set to
# an appropriate range given dataset_size and training budget.

# If your system does not use Python, you can save the JSON file (`template_path`),
# and use another programming language to submit the pipeline.
hpt_job_template_path, hpt_job_parameter_values = utils.get_builtin_algorithm_hyperparameter_tuning_job_pipeline_and_parameters(
      project=GCP_PROJECT,
      location=GCP_REGION,
      root_dir=hpt_job_root_dir,
      algorithm='wide_and_deep',
      target_column=target_column,
      prediction_type=prediction_type,
      transform_config=transform_config_path,
      training_fraction=training_fraction,
      validation_fraction=validation_fraction,
      test_fraction=test_fraction,
      data_source_csv_filenames=csv_filenames,
      study_spec_metrics=metrics,
      study_spec_parameters_override=parameters,
      max_trial_count=1,
      parallel_trial_count=1,
      max_failed_trial_count=0,
      training_machine_spec=training_machine_spec,
      dataflow_use_public_ips=dataflow_use_public_ips,
      dataflow_subnetwork=dataflow_subnetwork,
      run_evaluation=True,
)

hpt_job_id = f'automl-tabular-wide-and-deep-hpt-{uuid.uuid4()}'
# More info on parameters PipelineJob accepts:
# https://cloud.google.com/vertex-ai/docs/pipelines/run-pipeline#create_a_pipeline_run 
hpt_job = aiplatform.PipelineJob(
    display_name=hpt_job_id,
    template_path=hpt_job_template_path,
    job_id=hpt_job_id,
    pipeline_root=hpt_job_root_dir,
    parameter_values=hpt_job_parameter_values,
    enable_caching=False,
)

hpt_job.run()

In [ ]:
# Get model URI
wide_and_deep_hpt_pipeline_task_details = aiplatform.PipelineJob.get(hpt_job_id).gca_resource.job_detail.task_details
print('model uri:', get_model_uri(wide_and_deep_hpt_pipeline_task_details))
print('model artifacts:', get_model_artifacts_path(wide_and_deep_hpt_pipeline_task_details, 'get-best-hyperparameter-tuning-job-trial'))

## Notes about service account and permission

**By default no configuration is required**, if you run into any permission related issue, please make sure the service accounts above have the required roles:

|Service account email|Description|Roles|
|---|---|---|
|PROJECT_NUMBER-compute@developer.gserviceaccount.com|Compute Engine default service account|Dataflow Admin, Dataflow Worker, Storage Admin, Service Account User, BigQuery Admin, Vertex AI User|
|service-PROJECT_NUMBER@gcp-sa-aiplatform.iam.gserviceaccount.com|AI Platform Service Agent|Vertex AI Service Agent, Dataflow Admin, Service Account Token Creator|


1. Goto https://console.cloud.google.com/iam-admin/iam.
2. Check the 'Include Google-provided role grants' checkbox.
3. Find the above emails.
4. Grant the corresponding roles.

### Using data source from a different project
- For the BQ data source, grant both service accounts the 'BigQuery Data Viewer' role.
- For the CSV data source, grant both service accounts the 'Storage Object Viewer' role.
